In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [ ]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

In [ ]:
#Preprocess data
# Drop irrevelnt data
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data

In [ ]:
## encode categorical variables

label_encoder_gender = LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data

In [ ]:
# onehot encode Geography
from sklearn.preprocessing import OneHotEncoder
onr_hot_encoder=OneHotEncoder()
geo_encoder=onr_hot_encoder.fit_transform(data[['Geography']])
geo_encoder

In [ ]:
onr_hot_encoder.get_feature_names_out(['Geography'])

In [ ]:
geo_encoder_df = pd.DataFrame(geo_encoder.toarray(),columns=onr_hot_encoder.get_feature_names_out(['Geography']))
geo_encoder_df

In [ ]:
#combine one hot encode column with orginal data
data = pd.concat([data.drop('Geography',axis=1),geo_encoder_df],axis=1)
data.head()

In [ ]:
#save the encoder and scaler
with open('Label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onr_hot_encoder,file)

In [ ]:
#divide the dataset into indepent and dependent set

X= data.drop('Exited',axis=1)
y=data['Exited']

#split the data in training  and tetsing sets

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

#scale these features
Scaler=StandardScaler()
X_train=Scaler.fit_transform(X_train)
X_test=Scaler.transform(X_test)

In [ ]:
X_train

In [ ]:
with open('scaler.pkl','wb') as file:
    pickle.dump(Scaler,file)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [ ]:
#Build our ANN Model
model= Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), # 1st Hidden  layer connceted with input layer
    Dense(32,activation='relu'), # HL 2
    Dense(1,activation='sigmoid') #output layer
])

In [ ]:
model.summary()

In [ ]:
import tensorflow
opt= tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()
loss

In [ ]:
#compile the model

model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])

In [ ]:
#SETUP the tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
log_dir = "logs/fit" +datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)


In [ ]:
#Set up Early Stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)




In [ ]:
#Train the model
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

In [ ]:
model.save('model.h5')

In [ ]:
#Load tensorboard extension
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit